### Google Cloud Vision - Face Detection API ###

In [1]:
#!/usr/bin/env python

# Copyright 2015 Google, Inc
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Draws squares around faces in the given image."""
import argparse
import base64

from PIL import Image
from PIL import ImageDraw

from googleapiclient import discovery
import httplib2
from oauth2client.client import GoogleCredentials
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './auth.json'


# [START get_vision_service]
DISCOVERY_URL='https://vision.googleapis.com/$discovery/rest?version=v1'


def get_vision_service():
    credentials = GoogleCredentials.get_application_default()
    return discovery.build('vision', 'v1', credentials=credentials,
                           discoveryServiceUrl=DISCOVERY_URL)
# [END get_vision_service]


# [START detect_face]
def detect_face(face_file, max_results=4):
    """Uses the Vision API to detect faces in the given file.
    Args:
        face_file: A file-like object containing an image with faces.
    Returns:
        An array of dicts with information about the faces in the picture.
    """
    image_content = face_file.read()
    batch_request = [{
        'image': {
            'content': base64.b64encode(image_content).decode('UTF-8')
            },
        'features': [{
            'type': 'FACE_DETECTION',
            'maxResults': max_results,
            }]
        }]

    service = get_vision_service()
    request = service.images().annotate(body={
        'requests': batch_request,
        })
    response = request.execute()
    return response['responses'][0]['faceAnnotations']
# [END detect_face]

# [START print_landmarks]
def print_landmarks(face, image):
    """prints out landmark results for a face object"""
    coordinates = list()
    width = image.size[0]
    height = image.size[1]
    x_norm = y_norm = float(250); #relatively arbitrary, right now just approximated by cropped face size from ck dataset
    x_norm_ratio = x_norm/width;
    y_norm_ratio = y_norm/height;
    for landmark in face['landmarks']:
        # print landmark['type']
        # print 'x: ' + str(landmark['position']['x'])
        # print 'y: ' + str(landmark['position']['y'])
        # print 'z: ' + str(landmark['position']['z'])
        coordinates.append((landmark['position']['x'])*x_norm_ratio)
        coordinates.append((landmark['position']['y'])*y_norm_ratio)
        coordinates.append(landmark['position']['z'])
    return coordinates

    #print face['landmarks'][1]
# [END print_landmarks]

# def normalize_landmark(width, height, x_coord, y_coord, x_norm, y_norm): #norm - 250, 250
#     #return ()
#     return 0

# [START crop_face]
def crop_face(image, face, output_filename):
    """Crops a polygon around the faces, then saves to output_filename.
    Args:
      image: a file containing the image with the faces.
      faces: a list of faces found in the file. This should be in the format
          returned by the Vision API.
      output_filename: the name of the image file to be created, where the faces
          have polygons drawn around them.
    """
    im = Image.open(image)

    box = [(v.get('x', 0.0), v.get('y', 0.0)) for v in face['fdBoundingPoly']['vertices']]
    corners = box[0] + box[2]
    im = im.crop(corners)
    im.save('images/' + output_filename)
    return im
# [END crop_face]

# [START face_detection]
def face_detection(input_filename, output_filename, max_results):
    with open(input_filename, 'rb') as image:
        face = detect_face(image, 1)[0] #detects face in original image <- inefficient way to find crop region?
        face_crop = crop_face(image, face, output_filename) #crop original image to just facial region
        with open('images/' + output_filename, 'rb') as image_crop: #rerun facial detection on cropped face
            face = detect_face(image_crop, 1)[0]
        return print_landmarks(face, Image.open('images/'+output_filename))
        #print('Writing to file %s' % output_filename)
        # Reset the file pointer, so we can read the file again
        #image.seek(0)
        #highlight_faces(image, faces, output_filename)
# [END face_detection]


# if __name__ == '__main__':
#     parser = argparse.ArgumentParser(
#         description='Detects faces in the given image.')
#     parser.add_argument(
#         'input_image', help='the image you\'d like to detect faces in.')
#     parser.add_argument(
#         '--out', dest='output', default='out.jpg',
#         help='the name of the output file.')
#     parser.add_argument(
#         '--max-results', dest='max_results', default=4,
#         help='the max results of face detection.')
#     args = parser.parse_args()

#     main(args.input_image, args.output, args.max_results)


In [2]:
#test run
face_detection('./images/CK+/cohn-kanade-plus-images/S051/002/S051_002_00000010.png', "out.jpg", 1)

[75.81367015209126,
 83.23715874524714,
 -0.00054242183,
 180.04882129277567,
 78.53305703422053,
 14.456583,
 35.73074619771864,
 59.44246958174905,
 9.4364767,
 103.66621673003803,
 53.22846863117871,
 -14.290893,
 156.07673954372626,
 51.174287072243345,
 -7.1145382,
 216.32480988593156,
 53.97913403041825,
 33.93354,
 131.37735741444868,
 74.72743916349809,
 -15.821347,
 137.98660646387833,
 131.50543726235742,
 -58.335003,
 138.3024049429658,
 174.43837452471485,
 -38.048069,
 138.9680133079848,
 212.615427756654,
 -35.473053,
 93.38339733840304,
 198.56468631178709,
 -16.282583,
 177.31248098859314,
 194.01611216730038,
 -4.5451975,
 137.39018060836503,
 191.52877376425855,
 -32.174747,
 166.0429657794677,
 144.23876425855514,
 -11.593722,
 105.35092205323194,
 147.40304182509507,
 -20.421896,
 137.48161596958175,
 151.84475285171104,
 -35.026592,
 75.17877756653994,
 74.9302633079848,
 -6.5142503,
 98.85173954372624,
 84.36564163498099,
 3.1294875,
 74.64830228136881,
 92.174515

### angle features ### 

In [3]:
0. LEFT_EYE	Left eye.
1.RIGHT_EYE	Right eye.
2.LEFT_OF_LEFT_EYEBROW	Left of left eyebrow.
3.RIGHT_OF_LEFT_EYEBROW	Right of left eyebrow.
4.LEFT_OF_RIGHT_EYEBROW	Left of right eyebrow.
5.RIGHT_OF_RIGHT_EYEBROW	Right of right eyebrow.
6.MIDPOINT_BETWEEN_EYES	Midpoint between eyes.
7.NOSE_TIP	Nose tip.
8.UPPER_LIP	Upper lip.
9.LOWER_LIP	Lower lip.
10.MOUTH_LEFT	Mouth left.
11.MOUTH_RIGHT	Mouth right.
12.MOUTH_CENTER	Mouth center.
13.NOSE_BOTTOM_RIGHT	Nose, bottom right.
14.NOSE_BOTTOM_LEFT	Nose, bottom left.
15.NOSE_BOTTOM_CENTER	Nose, bottom center.
16.LEFT_EYE_TOP_BOUNDARY	Left eye, top boundary.
17.LEFT_EYE_RIGHT_CORNER	Left eye, right corner.
18.LEFT_EYE_BOTTOM_BOUNDARY	Left eye, bottom boundary.
19.LEFT_EYE_LEFT_CORNER	Left eye, left corner.
20.RIGHT_EYE_TOP_BOUNDARY	Right eye, top boundary.
21.RIGHT_EYE_RIGHT_CORNER	Right eye, right corner.
22.RIGHT_EYE_BOTTOM_BOUNDARY	Right eye, bottom boundary.
23.RIGHT_EYE_LEFT_CORNER	Right eye, left corner.
24.LEFT_EYEBROW_UPPER_MIDPOINT	Left eyebrow, upper midpoint.
25.RIGHT_EYEBROW_UPPER_MIDPOINT	Right eyebrow, upper midpoint.
26.LEFT_EAR_TRAGION	Left ear tragion.
27.RIGHT_EAR_TRAGION	Right ear tragion.
28.LEFT_EYE_PUPIL	Left eye pupil.
29.RIGHT_EYE_PUPIL	Right eye pupil.
30.FOREHEAD_GLABELLA	Forehead glabella.
31.CHIN_GNATHION	Chin gnathion.
32.CHIN_LEFT_GONION	Chin left gonion.
33.CHIN_RIGHT_GONION	Chin right gonion.

SyntaxError: invalid syntax (<ipython-input-3-c58310fb575e>, line 1)

In [4]:
I get a landmark

SyntaxError: invalid syntax (<ipython-input-4-338cae145a5a>, line 1)

In [5]:
#!/usr/bin/python

import os
from os import listdir
from cd import cd

def get_all_folders(folder_path):
    with cd(folder_path):
        all_folders = os.listdir('.')
        directory_names = []
        for directory in all_folders:
            if directory[0] == '.':
                continue
            else:
                directory_names.append(directory)
        return directory_names
    
labels = list()
features = list()

def grab_data(Folder):
    counter = 0
    
    ck_path = os.getcwd() + '/images/CK+/' + Folder + '/'
    s_folders = get_all_folders(ck_path)
    for folder in s_folders:
        subdir_path = ck_path + folder + '/'
        zero_folders = get_all_folders(subdir_path)
        for zero in zero_folders:
            img_level_path = subdir_path + zero + '/'
            imgs = get_all_folders(img_level_path)
            if Folder == 'Emotions': #LABELS
                if len(imgs) > 0: #has an emotion label
                    last_img_path = img_level_path + imgs[-1]
                    f = open(last_img_path, 'r')
                    emotion = f.read()
                    labels.append(emotion[3]) #emotion[3] is the number denoting emotion
                else: #has no emotion label
                    labels.append(-1)
            else: #FEATURES
                last_img_path = img_level_path + imgs[-1]
                if counter < len(labels): #TODO: CHECK THIS OFF BY ONE BULLSHIT
                    if labels[counter] != -1:
                        print counter
                        features.append(face_detection(last_img_path, str(counter) + ".jpg", 1))
                counter += 1
#                 if counter > 5: #because of bad internet
#                     return




In [6]:
grab_data('Emotions')
grab_data('cohn-kanade-plus-images')
temp_labels = list()
for label in labels: #clean temp labels
    if label != -1:
        temp_labels.append(label)
labels = temp_labels

0
2
4
6
7
8
9
10
11
12
13
14
15
17
18
22
23
24
25
26
29
30
31
32
34
35
36
37
38
40
42
43
47
48
49
51
54
55
56
58
60
61
63
66
70
71
72
73
74
75
77
78
80
82
84
85
86
89
90
91
93
95
96
97
98
100
101
102
103
105
106
107
108
110
113
114
117
118
119
120
123
124
125
126
127
128
129
130
132
133
134
135
136
138
139
141
142
143
144
146
147
148
149
151
153
154
155
157
158
159
160
162
163
164
167
168
169
170
171
173
174
175
180
181
182
187
188
189
191
192
194
197
198
200
201
205
206
207
212
214
217
220
221
222
224
225
229
230
231
232
236
237
241
243
246
248
249
251
252
253
254
255
256
259
260
262
263
266
267
268
269
270
271
272
273
275
276
278
279
280
283
284
286
287
292
293
294
295
297
298
299
302
304
307
308
309
312
313
314
317
318
320
325
326
332
349
351
353
355
357
361
366
367
369
372
374
375
376
383
388
389
394
395
397
402
403
408
409
412
416
417
422
423
424
426
429
437
439
444
452
456
458
461
462
463
467
468
469
470
471
472
473
476
477
479
480
481
482
484
485
486
492
494
495
496
497
499
502


In [7]:
## CONVERT TO FLOATS ##
import numpy as np

labels = np.array(labels).astype(np.float) #does it matter if the labels are strings or floats? what's the difference?
#print labels

features = np.array(features).astype(np.float)
print features

[[  7.49039760e+01   8.30994489e+01   1.55015050e-03 ...,   2.30113339e+02
    1.83858450e+02   1.13633720e+02]
 [  7.10013750e+01   8.52327887e+01  -1.73937260e-03 ...,   2.40550000e+02
    2.07707702e+02   9.99086380e+01]
 [  6.97876577e+01   8.13421929e+01  -9.44529950e-04 ...,   2.36836720e+02
    1.89456563e+02   9.12832720e+01]
 ..., 
 [  6.05348421e+01   7.99952588e+01   2.10058920e-03 ...,   2.17467281e+02
    1.93193500e+02   9.32072830e+01]
 [  7.25139441e+01   8.31724095e+01  -1.86232990e-03 ...,   2.30141464e+02
    2.02685329e+02   8.96114810e+01]
 [  7.53539794e+01   8.68704841e+01   1.41212260e-03 ...,   2.37296381e+02
    2.11071754e+02   6.83160550e+01]]


In [8]:
# import os
# if not os.path.exists('my_file'): numpy.savetxt('my_file', my_array)

# np.savetxt("ck_features_norm", features)
# np.savetxt("ck_labels", labels)
labels = np.loadtxt("ck_labels")
features = np.loadtxt("ck_features_norm")
features_no_norm = np.loadtxt("ck_features")

In [9]:
# you can now use np.loadtxt("") to load in some of the google api generated landmark arrays.
np.loadtxt("ck_features")

array([[  9.37797780e+01,   1.04040510e+02,   1.55015050e-03, ...,
          2.88101900e+02,   2.30190780e+02,   1.13633720e+02],
       [  8.06575620e+01,   9.68244480e+01,  -1.73937260e-03, ...,
          2.73264800e+02,   2.35955950e+02,   9.99086380e+01],
       [  7.78830260e+01,   9.11032560e+01,  -9.44529950e-04, ...,
          2.64309780e+02,   2.12191350e+02,   9.12832720e+01],
       ..., 
       [  6.90097200e+01,   9.11945950e+01,   2.10058920e-03, ...,
          2.47912700e+02,   2.20240590e+02,   9.32072830e+01],
       [  8.81769560e+01,   1.01137650e+02,  -1.86232990e-03, ...,
          2.79852020e+02,   2.46465360e+02,   8.96114810e+01],
       [  9.49460140e+01,   1.09456810e+02,   1.41212260e-03, ...,
          2.98993440e+02,   2.65950410e+02,   6.83160550e+01]])

In [10]:
### ARRAY LENGTH SANITY CHECK ###
print len(features)
print len(labels)

327
327


In [11]:
import numpy as np
import itertools

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))


angle_features = []
for x in range(len(labels)):
    angle_features.append([])
print angle_features
# def make_angle_features():
#     num = 0
#     LOWER_LIP = 9
#     MOUTH_LEFT = 10
#     MOUTH_RIGHT = 11
#     for landmarks in features_no_norm:
#         print "LOWER LIP: " + str(landmarks[LOWER_LIP*3]) + "," + str(landmarks[LOWER_LIP*3+1])
#         print "MOUTH_LEFT: " + str(landmarks[MOUTH_LEFT*3]) + "," + str(landmarks[MOUTH_LEFT*3+1])
#         print "MOUTH_RIGHT: " + str(landmarks[MOUTH_RIGHT*3]) + "," + str(landmarks[MOUTH_RIGHT*3+1])
#         #lowerlip to left (lowerlip x - left x, lowerlip y - left y)
#         v1 = (landmarks[LOWER_LIP*3]-landmarks[MOUTH_LEFT*3], landmarks[LOWER_LIP*3+1]-landmarks[MOUTH_LEFT*3+1])
#         #lowerlip to right
#         v2 = (landmarks[LOWER_LIP*3]-landmarks[MOUTH_RIGHT*3], landmarks[LOWER_LIP*3+1]-landmarks[MOUTH_RIGHT*3+1])
#         smile_angle = angle_between(v1, v2);
#         print "ANGLE BETWEEN: " + str(smile_angle * 180 / 3.1415)
#         angle_features.append([smile_angle])
#         print('---')
#     #for each feature vector in features:
# #     9.LOWER_LIP	Lower lip.
# #     10.MOUTH_LEFT	Mouth left.
# #     11.MOUTH_RIGHT	Mouth right.
        
def add_angle(vertex, left, right): #inputs are represented by the number corresponding to the landmark
    for i in range(len(features_no_norm)):
        landmarks = features_no_norm[i]
        vector1 = (landmarks[vertex*3]-landmarks[left*3], landmarks[vertex*3+1]-landmarks[left*3+1])
        vector2 = (landmarks[vertex*3]-landmarks[right*3], landmarks[vertex*3+1]-landmarks[right*3+1])
        angle_features[i].append(angle_between(vector1, vector2));

possible_angles = list(itertools.permutations([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                                  16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33], 3))
counter = 0


[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

In [12]:
# for angle in possible_angles[0:3000]: #crashes on 30,000
#     add_angle(angle[0], angle[1], angle[2])
#     print counter
#     counter += 1

add_angle(9, 10, 11)
add_angle(6, 2, 4)
add_angle(21, 20, 22)
    
angle_features = np.array(angle_features).astype(np.float)

In [13]:
#tools
def train_test_split(split): 
    features_train = angle_features[:split]
    labels_train = labels[:split]
    features_test = angle_features[split:]
    labels_test = labels[split:]
    return (features_train, labels_train, features_test, labels_test)

def num_of_label(val, labels):
    count = 0
    for label in labels:
        if label == val:
            count += 1
    return count

def print_accuracy(labels_actual, labels_predicted):
    correct = 0
    wrong = 0
    for index in range(len(labels_actual)):
        if labels_actual[index] == labels_predicted[index]:
            correct += 1
        else:
            wrong += 1
    print 'num correct: ' + str(correct)
    print 'num wrong: ' + str(wrong)
    print 'accuracy: ' + str(float(correct)/(correct + wrong))

In [14]:
from sklearn import svm

#X is a matrix of input examples [[x1.....xn],...[x1,..., xn]]
#Y is a vector of labels corresponding to emotion [1, 2, 8, ... yn]


def train(X, y, c, gammaval):
    clf = svm.SVC(decision_function_shape='ovo', C=c, gamma=gammaval)
#    clf = svm.SVC(kernel='linear', C=1)
#     clf = svm.LinearSVC()
    clf.fit(X,y)
    print 'Finished Training SVM'
    return clf

def test(clf, X):
    #Takes all examples to predict and returns an array of lables
    predictions = clf.predict(X)
    print 'Finished Predictions for SVM'
    return predictions

############ Sample usage ####################

#X = [[0,1], [3,4], [6,2], [1,2]]
#y = [1,2,3,4]
    
splits = train_test_split(250)
features_train = splits[0]
labels_train = splits[1]
features_test = splits[2]
labels_test = splits[3]

print features_train
clf = train(features_train, labels_train)
predictions = test(clf, features_test)

# print(clf.score(features_test, labels_test))

print "training labels: "
print labels_train

for val in range (0, 8):
    print "TRAIN: num of label " + str(val) + ": " + str(num_of_label(val, labels_train))

### ERROR ANALYSIS ### 
print "predictions: "
print predictions
print "actual: "
print labels_test

print_accuracy(labels_test, predictions)

[[ 2.11247924  2.1932017   2.77411122]
 [ 2.19416763  2.35346848  2.73458436]
 [ 2.16730822  2.34124827  2.75687824]
 [ 2.05300337  2.37138611  2.79251749]
 [ 2.23154664  2.22708178  2.69377591]
 [ 2.25880904  2.17706897  2.67614413]
 [ 2.39605262  2.14633197  2.68195649]
 [ 2.51193711  2.1040047   2.7158561 ]
 [ 2.1932467   2.21036814  2.66369605]
 [ 2.06678912  2.17524563  2.81814487]
 [ 2.2070383   2.2629727   2.72342873]
 [ 2.26197488  2.37383493  2.76141462]
 [ 2.54426524  2.36694579  2.7713701 ]
 [ 2.17433947  2.35392987  2.82938359]
 [ 2.36021852  2.17608996  2.52700154]
 [ 2.49230341  2.27742638  2.67566762]
 [ 2.27360601  2.27752936  2.68518369]
 [ 2.14681252  2.09590274  2.64052944]
 [ 2.52509876  2.15808459  2.79312863]
 [ 2.41669723  2.13601291  2.89611676]
 [ 1.84200649  2.1442396   2.90358964]
 [ 2.1332843   2.17962766  2.90188592]
 [ 2.35457013  2.31283615  2.76405678]
 [ 2.28790047  2.33040938  2.71380255]
 [ 2.4795464   2.29445084  2.74065439]
 [ 2.14854097  2.29730527

TypeError: train() takes exactly 4 arguments (2 given)

In [29]:
cval = [.1, 1, 10, 100, 1000]
gammas = [2**-7, 2**-6, 2**-5, 2**-4, 2**-3, 2**-2, 2**-1]
for c in cval:
    for gamma in gammas:
        clf = train(features_train, labels_train, c, gamma)
        predictions = test(clf, features_test)
        print_accuracy(labels_test, predictions)

Finished Training SVM
Finished Predictions for SVM
num correct: 10
num wrong: 67
accuracy: 0.12987012987
Finished Training SVM
Finished Predictions for SVM
num correct: 10
num wrong: 67
accuracy: 0.12987012987
Finished Training SVM
Finished Predictions for SVM
num correct: 10
num wrong: 67
accuracy: 0.12987012987
Finished Training SVM
Finished Predictions for SVM
num correct: 10
num wrong: 67
accuracy: 0.12987012987
Finished Training SVM
Finished Predictions for SVM
num correct: 10
num wrong: 67
accuracy: 0.12987012987
Finished Training SVM
Finished Predictions for SVM
num correct: 10
num wrong: 67
accuracy: 0.12987012987
Finished Training SVM
Finished Predictions for SVM
num correct: 10
num wrong: 67
accuracy: 0.12987012987
Finished Training SVM
Finished Predictions for SVM
num correct: 10
num wrong: 67
accuracy: 0.12987012987
Finished Training SVM
Finished Predictions for SVM
num correct: 10
num wrong: 67
accuracy: 0.12987012987
Finished Training SVM
Finished Predictions for SVM
num 

In [15]:
##LINEAR REGRESSION##
from sklearn import linear_model
#clf = linear_model.Ridge (alpha = .5)
clf = linear_model.LinearRegression()
clf.fit(features_train, labels_train)
# print("coefficient: ")
# clf.coef_
# print("predict: ")
test_results = clf.predict(features_test)
updated_results = list()
for result in test_results:
    updated_results.append(int(round(result)))
updated_results = np.array(updated_results).astype(np.float)
print updated_results
print labels_test

print_accuracy(labels_test, updated_results)


[ 5.  4.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  4.  4.  5.  4.  5.
  3.  4.  4.  4.  4.  5.  4.  4.  5.  5.  5.  6.  4.  5.  5.  5.  4.  5.
  5.  5.  5.  4.  5.  5.  5.  5.  5.  5.  5.  5.  4.  5.  4.  5.  4.  5.
  4.  4.  4.  4.  5.  5.  4.  4.  4.  4.  4.  5.  3.  4.  3.  4.  4.  5.
  3.  3.  3.  4.  4.]
[ 1.  7.  5.  1.  3.  5.  7.  1.  3.  5.  7.  1.  6.  3.  5.  7.  6.  5.
  3.  6.  4.  3.  5.  7.  1.  7.  5.  1.  5.  3.  7.  5.  7.  6.  1.  5.
  7.  6.  5.  4.  7.  5.  6.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
  2.  1.  4.  6.  1.  2.  4.  1.  2.  6.  1.  2.  4.  6.  2.  6.  1.  2.
  4.  6.  2.  1.  4.]
num correct: 16
num wrong: 61
accuracy: 0.207792207792


In [16]:
##RANDOM##
import random

num_examples = len(predictions)
random_labels = [random.randint(0,7) for p in range(0,num_examples)]
# print random_labels
# print labels_test
print_accuracy(labels_test, random_labels)

NameError: name 'predictions' is not defined

In [37]:
### CROSS VALIDATION ###
from sklearn import metrics
from sklearn import cross_validation

clf = svm.SVC(C=1000, decision_function_shape='ovo', gamma=2**-1, kernel='linear')
clf.fit(features_train, labels_train)
predictions = test(clf, features_test)

#clf = svm.SVC(kernel='linear', C=1)
scores = cross_validation.cross_val_score(clf, features, labels, cv=5)

scores   
predictions


Finished Predictions for SVM


array([ 7.,  5.,  5.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,
        1.,  5.,  7.,  5.,  5.,  1.,  1.,  5.,  5.,  5.,  5.,  5.,  1.,
        7.,  7.,  7.,  7.,  1.,  7.,  5.,  7.,  5.,  7.,  7.,  5.,  7.,
        7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  5.,  5.,
        5.,  7.,  1.,  5.,  5.,  1.,  7.,  7.,  1.,  1.,  1.,  1.,  1.,
        7.,  1.,  1.,  1.,  1.,  1.,  7.,  1.,  1.,  1.,  7.,  1.])